In [1]:
import pandas as pd, os, gc
import numpy as np
import math
import os
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from typing import List 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/My Drive/10Acad/Train.csv'
path_test = '/content/drive/My Drive/10Acad/Test.csv'

In [4]:
'''path = "../data/"
trains='Train.csv'
tests='Test.csv'''
    
train =pd.read_csv(path, parse_dates=['join_date'])
test = pd.read_csv(path_test, parse_dates=['join_date'])
    
    
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
    
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products_t, var_name = "PCODE", value_name="Label" )
    
    #Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE']
test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']
test.drop('Label', inplace=True, axis=1) 

In [5]:
print(train['occupation_code'].nunique())
print(test['occupation_code'].nunique())

233
187


In [6]:
train.sample(4)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,ID X PCODE
103840,RVN6KW6,2017-01-10,F,S,1984,748L,P4MD,90QI,7POT,0,RVN6KW6 X 7POT
607968,CT9EE8P,2015-01-04,M,M,1976,748L,BP09,56SI,ECY3,0,CT9EE8P X ECY3
312683,D0J0I84,2018-01-06,F,M,1980,30H5,0KID,T4MS,N2MW,0,D0J0I84 X N2MW
197606,82SO3AC,2020-01-01,F,S,1964,30H5,UBBX,T4MS,SOP4,0,82SO3AC X SOP4


In [7]:
test.sample(4)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
98826,6MCA3VC,2019-01-05,M,M,1983,94KC,2A7I,T4MS,LJR9,6MCA3VC X LJR9
82706,F9DGVKT,2017-01-07,M,M,1964,748L,SST3,56SI,PYUQ,F9DGVKT X PYUQ
203219,QAJCB6K,2015-01-01,F,M,1982,748L,0B60,JD7X,ECY3,QAJCB6K X ECY3
101890,A12OH6X,2019-01-07,M,M,1968,94KC,S9KU,90QI,N2MW,A12OH6X X N2MW


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611772 entries, 0 to 611771
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ID                        611772 non-null  object        
 1   join_date                 611730 non-null  datetime64[ns]
 2   sex                       611772 non-null  object        
 3   marital_status            611772 non-null  object        
 4   birth_year                611772 non-null  int64         
 5   branch_code               611772 non-null  object        
 6   occupation_code           611772 non-null  object        
 7   occupation_category_code  611772 non-null  object        
 8   PCODE                     611772 non-null  object        
 9   Label                     611772 non-null  int64         
 10  ID X PCODE                611772 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 51.3+ MB


In [9]:
#train['join_date'].value_counts()

In [10]:
#test['join_date'].value_counts()

In [11]:
#test.info()

In [12]:
train.fillna('2018-01-05', inplace=True)

In [13]:
train.isnull().sum()

ID                          0
join_date                   0
sex                         0
marital_status              0
birth_year                  0
branch_code                 0
occupation_code             0
occupation_category_code    0
PCODE                       0
Label                       0
ID X PCODE                  0
dtype: int64

In [14]:
test.fillna('2018-01-05', inplace=True)

In [15]:
#Computing the duration  of the client in the insurance company
#converting the 'Date' column  to date formate
train["join_date"] = pd.to_datetime(train["join_date"],format = "%Y-%m-%d")
test["join_date"] = pd.to_datetime(test["join_date"],format = "%Y-%m-%d")
today = datetime.date.today()
    
train["duration"] = today.year - train["join_date"].dt.year
#Computing the age  of the client
train["age"] = today.year - train["birth_year"]

test["duration"] = today.year - test["join_date"].dt.year
#Computing the age  of the client
test["age"] = today.year - test["birth_year"]

In [16]:
print(train.shape)
print(test.shape)

(611772, 13)
(210000, 12)


In [17]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


'''# Convert the following numerical labels from integer to float
train[["age", "duration","birth_year"]].values.astype(float)
test[["age", "duration","birth_year"]].values.astype(float)
    
# categorical features to be onverted to One Hot Encoding
cat_cols = ['branch_code', 'occupation_code','occupation_category_code','marital_status']
    
# One Hot Encoding conversion
train = pd.get_dummies(train, columns=cat_cols)
test = pd.get_dummies(test, columns=cat_cols)
    
# Label Encoder conversion
train["sex"] = le.fit_transform(train["sex"])
train["PCODE"] = le.fit_transform(train["PCODE"])
train["ID"] = le.fit_transform(train["ID"])

test["sex"] = le.fit_transform(test["sex"])
test["PCODE"] = le.fit_transform(test["PCODE"])
test["ID"] = le.fit_transform(test["ID"])'''


#Selecting the columns for label encoding.
categorical_columns = ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status']
#import preprocessing module
from sklearn.preprocessing import LabelEncoder

#le = LabelEncoder()

#Label Encoding for object to numeric conversion;train_data
for feat in categorical_columns:
    train[feat] = le.fit_transform(train[feat].astype(str))
    
#Label Encoding for object to numeric conversion;test_data
for feat in categorical_columns:
    test[feat] = le.fit_transform(test[feat].astype(str))

In [18]:
scaler = StandardScaler()
cols = ['age', 'duration', 'birth_year']
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.fit_transform(test[cols])

In [19]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,ID X PCODE,duration,age
0,3999,2019-01-02,0,1,0.802422,0,17,5,15,0,4WKQSBB X P5DA,-0.668801,-0.802422
1,10274,2019-01-06,0,1,0.159021,11,17,5,15,0,CP5S02H X P5DA,-0.668801,-0.159021
2,2388,2013-01-06,1,5,1.231356,3,176,1,15,0,2YKDILJ X P5DA,2.465359,-1.231356
3,2233,2019-01-08,1,1,1.124122,0,76,0,15,0,2S9E81J X P5DA,-0.668801,-1.124122
4,9326,2019-01-08,1,1,1.124122,3,148,5,15,0,BHDYVFT X P5DA,-0.668801,-1.124122


In [20]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,duration,age
0,4257,2018-01-12,1,2,0.463010,4,79,1,15,F86J5PC X P5DA,-0.146502,-0.463010
1,4834,2019-01-10,1,2,1.751982,0,103,1,15,H6141K3 X P5DA,-0.664178,-1.751982
2,7655,2020-01-01,0,7,-1.255620,11,18,5,15,RBAYUXZ X P5DA,-1.181853,1.255620
3,5729,2019-01-02,1,2,1.000081,4,18,5,15,KCBILBQ X P5DA,-0.664178,-1.000081
4,6141,2020-01-02,0,2,0.248181,11,3,5,15,LSEC1ZJ X P5DA,-1.181853,-0.248181


In [21]:
print(train.shape)
print(test.shape)

(611772, 13)
(210000, 12)


In [22]:
X = train.drop(columns=['join_date', 'ID X PCODE', 'Label'])
y = train['Label']
sampled_test = test.drop(columns=['join_date', 'ID X PCODE'])

In [23]:
'''from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)'''

'from sklearn.model_selection import train_test_split\nX_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=123)'

In [24]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 4)


for train_index,test_index in cv.split(X,y):
    X_train,X_test = X.loc[train_index],X.loc[test_index]
    y_train,y_test = y.loc[train_index],y.loc[test_index]

In [ ]:
### hyperparameter Tuning 


In [25]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV

clf = lgb.LGBMClassifier()

scores = []
# Fit the model on the training data
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Score the model on the validation data
score = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
        
scores.append(score)
mean_score = np.array(scores).mean()

In [26]:
print('Accuracy scores of the model: {:.2f}'.format(mean_score))
print('\n Classification report of the model')
print('--------------------------------------')
print(report)
print('\n Confusion Matrix of the model')
print('--------------------------------------')
print(conf_matrix)

Accuracy scores of the model: 0.96

 Classification report of the model
--------------------------------------
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     54541
           1       0.87      0.75      0.81      6636

    accuracy                           0.96     61177
   macro avg       0.92      0.87      0.89     61177
weighted avg       0.96      0.96      0.96     61177


 Confusion Matrix of the model
--------------------------------------
[[53802   739]
 [ 1633  5003]]


In [27]:
pred = model.predict(sampled_test)
#submission = pd.DataFrame({'ID X PCODE' : test['ID X PCODE'], 'Label':pred.index})
submission = pd.DataFrame({'ID X PCODE':test['ID X PCODE'],'Label':pred})

print('Pred_0: {}'.format(submission[submission['Label']==0].count()))
print('----------------------------------------------------------')
print('Pred_1: {}'.format(submission[submission['Label']==1].count()))

Pred_0: ID X PCODE    190973
Label         190973
dtype: int64
----------------------------------------------------------
Pred_1: ID X PCODE    19027
Label         19027
dtype: int64


In [28]:
submission.to_csv('/content/drive/My Drive/10Acad/data/submission16.csv', index=False)